In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import warnings
warnings.filterwarnings("ignore")
import pickle
import sys
main_path = r'../..'
sys.path.append(main_path)
from BayDS import *

In [2]:
main_folder = r'../../Data/sub'
model_folder = r'../../Snapshots/Stacking/3009'

In [17]:
import yaml
lb = yaml.load(open(f'{main_folder}/lb.yaml','r'), Loader=yaml.FullLoader)

In [56]:
oofs=None
predictions=None
for iexp, experiment in enumerate(lb):
    print(experiment)
    oof = pd.read_csv(f'{main_folder}/{experiment["oofFile"]}')
    pred = pd.read_csv(f'{main_folder}/{experiment["predictionFile"]}')
    oof.set_index('TransactionID')
    pred.set_index('TransactionID')
    if len(oof.index) != 590540:
        print (f"Skipping {experiment}")
        continue
    if oofs is None:
       oofs = pd.DataFrame(index=oof.index)
       predictions = pd.DataFrame(index=pred.index)
#     print(oof)
    oofs[f"s{iexp}_{experiment['score']}"] = oof['isFraud']
    predictions[f"s{iexp}_{experiment['score']}"] = pred['isFraud']

    

{'oofFile': 'oof_all_data_best_lgb_26.csv', 'predictionFile': 'prediction_all_data_best_lgb_26.csv', 'score': 0.9459}
Skipping {'oofFile': 'oof_all_data_best_lgb_26.csv', 'predictionFile': 'prediction_all_data_best_lgb_26.csv', 'score': 0.9459}
{'oofFile': 'oof_all_data_best_lgb_timefreq.csv', 'predictionFile': 'prediction_all_data_best_lgb_timefreq.csv', 'score': 0.9504}
{'oofFile': 'oof_all_data_best_lgb.csv', 'predictionFile': 'prediction_all_data_best_lgb.csv', 'score': 0.9484}
{'oofFile': 'oof_lgb_new_card_id_del_mean020_v3.csv', 'predictionFile': 'lgb_new_card_id_del_mean020_v3.csv', 'score': 0.947}
Skipping {'oofFile': 'oof_lgb_new_card_id_del_mean020_v3.csv', 'predictionFile': 'lgb_new_card_id_del_mean020_v3.csv', 'score': 0.947}
{'oofFile': 'oof_best_submit_09469.csv', 'predictionFile': 'submission_09469.csv', 'score': 0.9469}
{'oofFile': 'stacked_oof.csv', 'predictionFile': 'stacked.csv', 'score': 0.939}
{'oofFile': 'oof_result_pca.csv', 'predictionFile': 'lgb_pca.csv', 'scor

In [52]:
oofs.columns
for col in oofs.columns:
    print(col, oofs[col].mean(), predictions[col].mean())

s1_0.9504 0.02265162141866039 0.021841851123379302
s2_0.9484 0.02338705009684399 0.02252622090485728
s4_0.9469 0.026085073017210213 0.02271503190580545
s5_0.939 0.03244336042803625 0.0327943665788628
s6_0.9498 0.02274679991623929 0.021454021766264816
s7_0.9511 0.029589353552032082 0.027386008221827688
s8_0.9514 0.03583654856328075 0.03695540336835534
s9_0.9512 0.026460130397639534 0.02405884462802773
s10_0.9476 0.022057325574929676 0.020249746314245712
s11_0.9512 0.0356167768290346 0.03738436684876621
s12_0.9464 0.02029689001643901 0.019229494973673256
s13_0.9464 0.03563930771402968 0.034713472379135586


In [57]:
oofs.shape

(590540, 12)

In [58]:
predictions.shape

(506691, 12)

In [42]:
y = pd.read_pickle(r'e:\Kaggle\data\y.pkl')

In [43]:
n_fold = 5
folds = KFold(n_splits=n_fold)

In [44]:
from BayDS.lib.training import *

In [48]:
for col in oofs.columns:
    print(col,fast_auc(y,oofs[col]))

s1_0.9504 0.9337790683794884
s2_0.9484 0.9310257339819353
s4_0.9469 0.9331306395823962
s5_0.939 0.9401843734290808
s6_0.9498 0.9371002686597802
s7_0.9511 0.9492520152989988
s8_0.9514 0.9534682076165975
s9_0.9512 0.9499705540891189
s10_0.9476 0.9384990946131712
s11_0.9512 0.9524674067457763
s12_0.9464 0.9278023612986986
s13_0.9464 0.9359445800665418


## LogRegression stacking

In [59]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
train_options = {
    "model_type":'sklearn',
    'model':model,
    'folds': folds,
    "params": {},
    "eval_metric":'auc',
    'averaging': 'usual',
    'splits': n_fold,
    'n_jobs': -1,
    'groups': None
}

from sklearn import preprocessing
standart_scaler = preprocessing.StandardScaler()
X = pd.DataFrame(standart_scaler.fit_transform(oofs), index=oofs.index, columns=oofs.columns)
result_dict_logreg = train_model_classification_vb(X=X, X_test=predictions, y=y, **train_options)

Fold 1 started at Tue Oct  1 22:15:12 2019
Fold 0. auc: 0.9472.

Fold 2 started at Tue Oct  1 22:15:15 2019
Fold 1. auc: 0.9646.

Fold 3 started at Tue Oct  1 22:15:18 2019
Fold 2. auc: 0.9630.

Fold 4 started at Tue Oct  1 22:15:20 2019
Fold 3. auc: 0.9698.

Fold 5 started at Tue Oct  1 22:15:23 2019
Fold 4. auc: 0.9611.

CV mean score: 0.9611, std: 0.0075.


In [61]:
sub = pd.read_csv(f'../../data/sample_submission.csv')
sub['isFraud'] = result_dict_logreg['prediction']
sub.to_csv(f'{model_folder}/stacked_0110.csv', index=False)

## LightGBM Stacking (bad)

In [11]:


params = {
    'learning_rate': 0.001,
    'num_leaves': 20,
    'max_depth': 1,
    'min_child_weight': 10,
    'lambda_l1':2,
    'lambda_l2':3,
    'min_data_in_leaf' :10,
    'min_sum_hessian_in_leaf' : 0.0001,
    'bagging_fraction' : 0.8,
    'max_bin': 12,
    'feature_fraction' : 0.9,
    'bagging_freq' : 100,
    'min_gain_to_split': 0.1 }

train_options = {
    "model_type":'lgb',
    "params": params,
    "eval_metric":'auc',
    'early_stopping_rounds': 500,
    'n_estimators': 5000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
}


result_dict_lgb = train_model_classification(X=oof, X_test=prediction, y=y, params=params, folds=folds,
                                         model_type=train_options['model_type'], 
                                         eval_metric=train_options['eval_metric'],
                                         plot_feature_importance=True,
                                         verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                         n_estimators=train_options['n_estimators'], 
                                         averaging=train_options['averaging'],
                                         n_jobs=-1, groups=None)

Fold 1 started at Fri Sep  6 22:32:11 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.109891	training's auc: 0.906504	valid_1's binary_logloss: 0.0839437	valid_1's auc: 0.899767
Early stopping, best iteration is:
[102]	training's binary_logloss: 0.139285	training's auc: 0.906217	valid_1's binary_logloss: 0.102694	valid_1's auc: 0.899856
Fold 2 started at Fri Sep  6 22:33:04 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.101532	training's auc: 0.903072	valid_1's binary_logloss: 0.1142	valid_1's auc: 0.898168


KeyboardInterrupt: 

## Keras Stacking 

In [16]:
def StackModel_maker():
    k.clear_session()
    
    numerical_inputs = Input(shape=[oof.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(50, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(10, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = BatchNormalization()(x)    
    
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 16384,
    'epochs': 200,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [17]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)
# StackModel_maker().save(f'{model_folder}/keras.mdl')

In [18]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU':4}, log_device_placement=False) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [19]:
gc.collect()
result_dict_keras = train_model_classification(model=StackModel_maker, 
                                             X=oof,
                                             X_test=prediction,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Thu Sep 12 20:29:35 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/200
472432/472432 [==============================] - ETA: 15s - loss: 0.68 - ETA: 3s - loss: 0.6810 - ETA: 2s - loss: 0.679 - ETA: 1s - loss: 0.676 - ETA: 0s - loss: 0.673 - ETA: 0s - loss: 0.670 - ETA: 0s - loss: 0.669 - ETA: 0s - loss: 0.667 - ETA: 0s - loss: 0.665 - 1s 2us/step - loss: 0.6646 - val_loss: 0.6248
Epoch 2/200
472432/472432 [==============================] - ETA: 0s - loss: 0.647 - ETA: 0s - loss: 0.645 - ETA: 0s - loss: 0.644 - ETA: 0s - loss: 0.641 - ETA: 0s - loss: 0.640 - ETA: 0s - loss: 0.637 - ETA: 0s - loss: 0.636 - ETA: 0s - loss: 0.635 - 1s 1us/step - loss: 0.6335 - val_loss: 0.6260
Epoch 3/200
472432/472432 [==============================] - ETA: 0s - loss: 0.622 - ETA: 0s - loss: 0.619 - ETA: 0s - loss: 0.617 - ETA: 0s - loss: 0.614 - ETA: 0s - loss: 0.614 - ETA: 0s - loss: 0.612 - ETA: 0s - loss: 0.611 - ETA: 0s - loss: 0.609 - 1s 1us/step - loss: 0.6083 - 

472432/472432 [==============================] - ETA: 0s - loss: 0.121 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.117 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.115 - 1s 1us/step - loss: 0.1160 - val_loss: 0.1013
Epoch 53/200
472432/472432 [==============================] - ETA: 0s - loss: 0.118 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.115 - 1s 1us/step - loss: 0.1150 - val_loss: 0.0982
Epoch 54/200
472432/472432 [==============================] - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.112 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - 1s 1us/step - loss: 0.1134 - val_loss: 0.0969
Epoch 55/200
472432/472432 [=============

472432/472432 [==============================] - ETA: 0s - loss: 0.089 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - 1s 1us/step - loss: 0.0882 - val_loss: 0.0620
Epoch 105/200
472432/472432 [==============================] - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.087 - 1s 1us/step - loss: 0.0878 - val_loss: 0.0620
Epoch 106/200
472432/472432 [==============================] - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.088 - 1s 1us/step - loss: 0.0881 - val_loss: 0.0623
Epoch 107/200
472432/472432 [==============================] - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.088 - 

472432/472432 [==============================] - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.083 - 1s 1us/step - loss: 0.0838 - val_loss: 0.0594
Epoch 157/200
472432/472432 [==============================] - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.084 - 1s 1us/step - loss: 0.0837 - val_loss: 0.0594
Epoch 158/200
472432/472432 [==============================] - ETA: 0s - loss: 0.079 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.083 - 1s 1us/step - loss: 0.0837 - val_loss: 0.0593
Epoch 159/200
472432/472432 [==============================] - ETA: 0s - loss: 0.077 - ETA: 0s - loss: 0.080 - 

472432/472432 [==============================] - ETA: 0s - loss: 0.470 - ETA: 0s - loss: 0.470 - ETA: 0s - loss: 0.469 - ETA: 0s - loss: 0.466 - ETA: 0s - loss: 0.467 - ETA: 0s - loss: 0.467 - ETA: 0s - loss: 0.465 - ETA: 0s - loss: 0.464 - ETA: 0s - loss: 0.463 - ETA: 0s - loss: 0.462 - 1s 1us/step - loss: 0.4623 - val_loss: 0.4341
Epoch 9/200
472432/472432 [==============================] - ETA: 0s - loss: 0.453 - ETA: 0s - loss: 0.454 - ETA: 0s - loss: 0.450 - ETA: 0s - loss: 0.450 - ETA: 0s - loss: 0.448 - ETA: 0s - loss: 0.446 - ETA: 0s - loss: 0.444 - ETA: 0s - loss: 0.443 - ETA: 0s - loss: 0.442 - ETA: 0s - loss: 0.440 - 1s 1us/step - loss: 0.4404 - val_loss: 0.4129
Epoch 10/200
472432/472432 [==============================] - ETA: 0s - loss: 0.423 - ETA: 0s - loss: 0.423 - ETA: 0s - loss: 0.423 - ETA: 0s - loss: 0.423 - ETA: 0s - loss: 0.422 - ETA: 0s - loss: 0.421 - ETA: 0s - loss: 0.420 - ETA: 0s - loss: 0.419 - 1s 1us/step - loss: 0.4192 - val_loss: 0.3928
Epoch 11/200
47243

...

472432/472432 [==============================] - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - 1s 1us/step - loss: 0.0823 - val_loss: 0.0780
Epoch 150/200
472432/472432 [==============================] - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - 1s 1us/step - loss: 0.0821 - val_loss: 0.0781
Epoch 151/200
472432/472432 [==============================] - ETA: 0s - loss: 0.080 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.081 - 1s 1us/step - loss: 0.0816 - val_loss: 0.0780
Epoch 152/200
472432/472432 [==============================] - 

118108/118108 [==============================] - ETA:  - 0s 1us/step
Fold 3. auc: 0.9598.
Fold 5 started at Thu Sep 12 20:38:22 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/200
472432/472432 [==============================] - ETA: 16s - loss: 0.72 - ETA: 3s - loss: 0.7238 - ETA: 1s - loss: 0.719 - ETA: 1s - loss: 0.714 - ETA: 0s - loss: 0.709 - ETA: 0s - loss: 0.705 - ETA: 0s - loss: 0.702 - ETA: 0s - loss: 0.697 - 1s 3us/step - loss: 0.6930 - val_loss: 0.6585
Epoch 2/200
472432/472432 [==============================] - ETA: 0s - loss: 0.656 - ETA: 0s - loss: 0.656 - ETA: 0s - loss: 0.656 - ETA: 0s - loss: 0.653 - ETA: 0s - loss: 0.650 - ETA: 0s - loss: 0.647 - ETA: 0s - loss: 0.645 - ETA: 0s - loss: 0.643 - ETA: 0s - loss: 0.641 - ETA: 0s - loss: 0.639 - 1s 1us/step - loss: 0.6390 - val_loss: 0.6216
Epoch 3/200
472432/472432 [==============================] - ETA: 0s - loss: 0.619 - ETA: 0s - loss: 0.614 - ETA: 0s - loss: 0.615 - ETA: 0s - loss: 0.614 - ETA: 0s - l

472432/472432 [==============================] - ETA: 0s - loss: 0.111 - ETA: 0s - loss: 0.109 - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.111 - ETA: 0s - loss: 0.111 - ETA: 0s - loss: 0.111 - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.110 - 1s 1us/step - loss: 0.1105 - val_loss: 0.1043
Epoch 53/200
472432/472432 [==============================] - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.111 - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.109 - ETA: 0s - loss: 0.108 - ETA: 0s - loss: 0.109 - ETA: 0s - loss: 0.109 - 1s 1us/step - loss: 0.1089 - val_loss: 0.1036
Epoch 54/200
472432/472432 [==============================] - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.109 - ETA: 0s - loss: 0.109 - ETA: 0s - loss: 0.107 - ETA: 0s - loss: 0.107 - ETA: 0s - loss: 0.108 - ETA: 0s - loss: 0.107 - ETA: 0s - loss: 0.107 - 1s 1us/step - loss: 0.1075 - val_loss: 0.1016
Epoch 55/200
472432/472432 [==============================] - ETA: 0s - loss: 0.108 - ETA: 0s - loss: 0.106 - ETA

472432/472432 [==============================] - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - 1s 1us/step - loss: 0.0859 - val_loss: 0.0797
Epoch 105/200
472432/472432 [==============================] - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - 1s 1us/step - loss: 0.0857 - val_loss: 0.0794
Epoch 106/200
472432/472432 [==============================] - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - 1s 1us/step - loss: 0.0861 - val_loss: 0.0795
Epoch 107/200
472432/472432 [==============================] - ETA: 0s - loss: 0.082 - 

Epoch 157/200
472432/472432 [==============================] - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.080 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - 1s 1us/step - loss: 0.0830 - val_loss: 0.0771
Epoch 158/200
472432/472432 [==============================] - ETA: 0s - loss: 0.079 - ETA: 0s - loss: 0.081 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - 1s 1us/step - loss: 0.0834 - val_loss: 0.0769
Epoch 159/200
472432/472432 [==============================] - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.083 - 1s 1us/step - loss: 0.0839 - val_loss: 0.0772
Epoch 160/200
472432/472432 [====================

In [21]:
sub = pd.read_csv(f'../../data/sample_submission.csv')
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv(f'{model_folder}/stacked_keras.csv', index=False)

In [22]:
import pickle
with open(f'{model_folder}/results_dict_stacked_keras.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)